# Agenda

1. Modules + packages
2. Basic objects
    - `class`
    - What happens when we define a class? What is a class, anyway?
    - Attributes -- the core of objects in Python
    - Attribute lookup via ICPO
    - Magic methods
    - Inheritance
    - Methods vs. functions
    - Properties
    - Descriptors

# Modules

Modules do two distinct things in Python:

1. They provide us with a library system, so that we can write code once and then use it many times. We can reuse our own code, and others can use our code, and we can use other people's code.
2. They provide us with namespaces.

# We use `import` to load a module

When we use `import`, we're really doing two or three things:

- Python loads the file associated with the name we're importing (this happens once)
- Python creates a module object based on the file, and stores it
- We get a global varible referring to that module object

Note that when we say `import modname`, we are not putting `modname` in quotes! We are not giving a path to a file, or anything else that could be interpeted as a string.We're giving the name of the variable we want to define.

In [1]:
import random

In [2]:
# Where does Python look for the "random" module? In "random.py". 
# Python looks, one by oine, through every directory in sys.path, and the first place it finds random.py, that file is loaded.

import sys
sys.path

['/Users/reuven/Courses/Current/Cisco-2024-07July-advanced',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python312.zip',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/lib-dynload',
 '',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/site-packages']

In [3]:
random

<module 'random' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/random.py'>

If you set the environment variable PYTHONPATH, that can contain the names of one or more directories where Python should also look for modules. Those will go after the standard library, but before site-packages.

# Caching of modules

The first time we say `import random`, Python finds the file, loads it, stores it in memory, and defines the variable.

The second and other times we say `import random`, Python sees that we already loaded it, and doesn't load it again. Rather, it just defines the variable to refer to the module object.

Where does Python cache this? In a dictionary, `sys.modules`. The keys to this dict are strings, the names of the modules, and the values are the module objects.

In [4]:
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from '/Users/reuven/.pyenv/versi

In [5]:
dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_ONE',
 '_Sequence',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_accumulate',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_fabs',
 '_floor',
 '_index',
 '_inst',
 '_isfinite',
 '_lgamma',
 '_log',
 '_log2',
 '_os',
 '_pi',
 '_random',
 '_repeat',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'binomialvariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randbytes',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

In [6]:
# if I say `import random` a second time, Python sees that it has that module ('random' in sys.modules),
# and so it just has to say

random = sys.modules['random']   # this defines the variable for the module object

# Variations on `import`

- `import MODNAME` -- loads the module (if needed), assigns the variable `MODNAME` to the module object. `MODNAME.py` is the filename that Python will look for in `sys.path`.
- `from MODNAME import NAME` -- loads the module (if needed), it assigns the variable `NAME` to `sys.modules[MODNAME].name`. It doesn't define `MODNAME` as a variable. We do this for convenience, so that we can write shorter variable/method names in our programs.
- `import MODNAME as ALIAS` -- this does the same as `import MODNAME`, but the variable that is assigned is `ALIAS`, not `MODNAME`.
- `from MODNAME import NAME as ALIAS` -- this does the same as `from .. import`, but only assigns one global variable, and it's `ALIAS`, referring to the individual attribute `NAME` on the module object.
- `from MODNAME import *` -- this imports *all* of the names in `MODNAME` as global variables into the current module. **NEVER EVER EVER EVER EVER EVER EVER EVER DO THIS!!**

In [7]:
import time

In [8]:
time

<module 'time' (built-in)>

In [9]:
from random import randint

In [ ]:
# from numpy import *
# from pandas import *
# from random import *
# from argparse import *

# __version__

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "pandas" that refers to sys.modules['pandas']

import pandas   

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "pd" that refers to sys.modules['pandas']

import pandas as pd

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "Series" that refers to sys.modules['pandas'].Series

from pandas import Series

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it runs a for loop over every name in sys.modules['pandas'], and defines a variable for each one

from pandas import *

for one_name in dir(sys.modules['pandas']):
    globals()[one_name] = getattr(sys.modules['pandas'], one_name)    

In [10]:
from pandas import elephant

ImportError: cannot import name 'elephant' from 'pandas' (/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/site-packages/pandas/__init__.py)

In [11]:
import mymod

In [12]:
mymod

<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [14]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [15]:
mymod.__file__

'/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'

In [16]:
mymod.__name__

'mymod'

In [17]:
import mymod

In [18]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [20]:
from importlib import reload

In [21]:
reload(mymod)  # this deletes sys.modules['mymod'], so that we can import it again

<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [22]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'hello',
 'x',
 'y']

In [23]:
mymod.x   # x is an attribute on the mymod object

10

In [24]:
mymod.y

[10, 20, 30]

In [25]:
mymod.hello('world')

'Hello, world!'

In [26]:
reload(mymod)

Hello
Goodbye


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [27]:
reload(mymod)

Hello from mymod
Goodbye from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [28]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'hello',
 'x',
 'y']

In [29]:
reload(mymod)

Hello from mymod
Goodbye from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

# `__name__` and `'__main__'`

The `__name__` variable always exists in Python, and it always contains a string indicating the current namespace. If you are in a module that was loaded with `import`, then the value of `__name__` will be the string of the file that was loaded. For example, when we said `import mymod`, the value of `__name__` inside of the module was the string `'mymod'`.

But the first module/program to be loaded into Python when it starts up has a special name, the string `'__main__'`. Don't confuse this with the `main` function in C or other programming languages! This simply means that we're in the first program to be loaded, and that no one imported us.

- `__name__` is a variable
- `'__main__'` is a string value

So what? We can then test the value of `__name__` in our module. If it's `'__main__'`, then we know the module is being run as a program, and not be imported as a module.

We can then have a module that also offers interactive features. The module's defintions will always be loaded, but only the stuff after the `if __name__ == '__main__'` will be executed when we're interactive.

In [30]:
reload(mymod)

Hello from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

# What's the use case for `if __name__ == '__main__'`?

- Run automated tests
- Start up a server
- Have the module demo its capabilities
- Offer interactive services based on the module's functionality

Above that line, you define variables, classes, and functions. Those will always be defined, whether it's run interactively or via `import`. But the stuff below that line will only be run interactively, when you execute the module as a program.

# Exercise: Menu function

Create a module, `menu.py`, which defines a single function, `menu`. This function takes any number of string arguments. Those are the options that the user can choose from. The idea is that someone writing a larger program wants to force the user to choose from among several predefined options. They can call `menu('a', 'b', 'c')`, and the user will be asked to enter a, b, or c.

The user's choice will be returned to the caller.

You should be able to write code like this:

```python
import menu

user_choice = menu.menu('a', 'b', 'c')   # user will be asked to choose from a, b, or c; their choice will be returned + assigned
print(f'User chose {user_choice}')
```

Also make it possible to execute the module from the command line. In such a case, it'll call `menu` with x, y, and z, and ask the user to choose from them. The user's choice will be displayed.

In [32]:
reload(menu)

user_choice = menu.menu('a', 'b', 'c')   # user will be asked to choose from a, b, or c; their choice will be returned + assigned
print(f'User chose {user_choice}')


Choose (a/b/c):  q


q is not a valid option; try again


Choose (a/b/c):  r


r is not a valid option; try again


Choose (a/b/c):  abc


abc is not a valid option; try again


Choose (a/b/c):  b


User chose b
